In [34]:
import  numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, classification_report
from  sklearn.model_selection import KFold



In [35]:
#PRE-PROCESSAMENTO
transform = transforms.Compose([
    transforms.Grayscale(),         #garante que as imagens são a preto e branco
    transforms.Resize((48, 48)),    #garante que têm dimensão 48x48
    transforms.ToTensor(),          #converte para tensores pytorch (1,48,48)
    transforms.Normalize((0.5,), (0.5,))  # normaliza pixels entre -1 e 1 (mean = 0.5, std = 0.5)
])

# "SPLIT" DATASET (definimos o treino e o teste):
#ImageFolder → usa os nomes das pastas como labels
train_dataset = datasets.ImageFolder( root="fer2013/train", transform=transform)
test_dataset = datasets.ImageFolder( root="fer2013/test", transform=transform)

#LOAD data
test = DataLoader( test_dataset, batch_size=64, shuffle=False)

labels correspondem ás emoçoes ou seja

imagens -> x

labels -> y

In [ ]:
#VERIFICAR SE ESTÁ TUDO OK
images, labels = next(iter(train)) #AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
print(images.shape)  # deve dar: [64, 1, 48, 48]
print(labels.shape)  # deve dar: [64]
print(images.min(), images.max())  # já está normalizado para -1 a 1


torch.Size([64, 1, 48, 48])
torch.Size([64])
tensor(-1.) tensor(1.)


0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [33]:
print(f"Labels values (y): \n {labels}")
print(f"image vector (x): \n {images[0,0,:5,:5]}") # mostra os primeiros 5x5 pixels da primeira imagem

Labels values (y): 
 tensor([6, 3, 5, 5, 0, 3, 3, 3, 2, 5, 0, 3, 3, 3, 2, 4, 2, 3, 0, 0, 0, 3, 2, 4,
        0, 4, 5, 0, 2, 5, 3, 1, 4, 2, 5, 4, 3, 2, 4, 4, 3, 4, 3, 6, 4, 4, 0, 0,
        3, 3, 4, 3, 6, 2, 3, 4, 5, 6, 3, 3, 0, 4, 6, 4])
image vector (x): 
 tensor([[ 0.9294,  0.5765, -0.2549, -0.1059, -0.1294],
        [ 0.9137,  0.7020, -0.0196, -0.0431, -0.1137],
        [ 0.8980,  0.6784, -0.0431,  0.0824, -0.1216],
        [ 0.9373,  0.5529, -0.2627,  0.0353, -0.2471],
        [ 0.9608,  0.5294, -0.5216, -0.1529, -0.3882]])


In [ ]:
def split_train_val(train_dataset, val_ratio, batch_size, seed):
    train_size = int((1 - val_ratio) * len(train_dataset))
    val_size = len(train_dataset) - train_size

    train_subset, val_subset = random_split(train_dataset, [train_size, val_size], 
                                            generator = torch.Generator().manual_sedd(seed))

    train = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    return train, val

In [ ]:
#do model CNN
num_emotions = 7
lr = 0.0005
num_epochs = 300
dropout = 0.1 
batch_size = 64

out_channels = nº filtros
kenerl = tamanho do filtro
filtro = conjunto de pesos aprendidos

In [ ]:
class CNN(nn.Module):
    #def __init__(self, num_classes = num_emotions):
    def __init__(self, num_classes = num_emotions, dropout_prob = dropout): #AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
        super(CNN, self).__init__()
        #1ª camada convolucional
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)   # (1,48,48) → (16,48,48)
        #2ª camada convolucional
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)  # (16,24,24) → (32,24,24)                         
        self.pool = nn.MaxPool2d(2,2)  # halves spatial size
        self.dropout = nn.Dropout(dropout_prob)
        
        #fully connected
        self.fc1 = nn.Linear(32 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):      # input x shape: (batch_size, 1, 48, 48)
        x = self.pool(F.relu(self.conv1(x)))  # conv1 + relu + pool: (batch,16,48,48) → (batch,16,24,24)
        x = self.pool(F.relu(self.conv2(x)))  # conv2 + relu + pool: (batch,32,24,24) → (batch,32,12,12)
        x = torch.flatten(x, 1)               # flatten except batch dim
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)                       #output logits
        return x

In [ ]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Bons candidatos para tuning:

-lr
-dropout
-nº de filtros por layer
-nº de camadas


-kernel size (às vezes)????? + padding tem valores padrao é ok

In [ ]:
#CROSS VALIDATION
def cross_validation_cnn(
    train_dataset,
    num_emotions,
    device,
    k_folds=5,
    patience = 5,
    batch_size=64,
    num_epochs=50,
    lr=1e-3,
    dropout_prob= 0.1,
):
    fold_results = []
    
    for run in range(k_folds):
        print(f"\n===== Fold {run+1}/{k_folds} =====")
        
        #split train - val
        seed = 42 + run   #“Different random seeds were used at each fold to ensure variability in the train–validation splits.”
        val_ratio = 0.2

        train, val = split_train_val(train_dataset, val_ratio, batch_size, seed)

    
        #MODEL (novo em cada k-fold)
        model = CNN (num_classes = num_emotions, dropout_prob = dropout_prob).to(device)
        criterion = nn.CrossEntropyLoss()    #for multi-class classification
        optimizer = optim.Adam(model.parameters(), lr = lr)

        #early stopping (por fold)
        best_val_loss = float("inf") #infinito positivo, para ser maior que qualquer loss possível
        best_val_acc = 0
        counter = 0


        for epoch in range(num_epochs):
            #----TRAIN----
            model.train()
            train_loss = 0
            correct = 0
            total = 0
            
            for images, labels in train:  #images = batch_x, labels = batch_y
                images, labels = images.to(device), labels.to(device)

                logits = model(images)    #logits =outputs
                loss = criterion(logits, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * images.size(0)
                _, preds = torch.max(logits, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            train_loss /= total
            train_acc = correct / total
        
            #----VALIDATION----
            model.eval()
            val_loss = 0
            correct = 0
            total = 0
    
            with torch.no_grad():
                for images, labels in val:
                    images, labels = images.to(device), labels.to(device)

                    logits = model(images)
                    loss = criterion(logits, labels)

                    val_loss += loss.item() * images.size(0)
                    _, preds = torch.max(logits, 1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)

                val_loss /= total
                val_acc = correct / total

                print(
                    f"Epoch [{epoch+1}/{num_epochs}] "
                    f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
                    f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
                #----EARLY STOPPING----  
                if val_loss < best_val_loss:
                   best_val_loss = val_loss
                   best_val_acc = val_acc
                   counter = 0
            
                else:
                    counter += 1
                    if counter >= patience:  #5 = patience-> hyperparameter
                       print("Early stopping triggered")
                       break

        fold_results.append(best_val_acc)
    
    mean_acc = sum(fold_results) / len(fold_results)
    print(f"\nMean validation accuracy over {k_folds} folds: {mean_acc:.4f}")

    return mean_acc     

na função de cima não estou a guardar o best model por fold apenas o best-val_acc.... os pesos não ficam guardados. ok para tuning mas para o futuro temos de ver se não tenho de acrescentar.

In [ ]:
#hyperparameters tunning
for lr in [0.001, 0.005, 1e-4]:
    for dropout in [0.0, 0.1, 0.2]:
        mean_acc = cross_validation_cnn(
            train_dataset,
            num_emotions,
            device,
            lr=lr,
            dropout_prob =dropout)


In [ ]:
#TESTE THE MODEL
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images_x, labels_y in test:
        images_x = images_x.to(device)
        labels_y = labels_y.to(device)
        logits = model(images_x)
        _, preds = torch.max(logits, 1)
        
        #all_preds.extend(preds.cpu().numpy())    -- converte para numpy pior se quiser continuar a usar pytorch/GPU
        all_preds.append(preds)
        all_labels.append(labels)  

# concatenar todos os batches em um tensor só
all_preds = torch.cat(all_preds).cpu()
all_labels = torch.cat(all_labels).cpu()

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")       


: 

In [ ]:
#CLASSIFICATION REPORT (stor way)
print("Classification Report:")
print(classification_report(all_labels, all_preds))

: 

In [ ]:
#CLASSIFICATION REPORT (CHAT WAY)
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=[
    'Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'
]))

: 

In [ ]:
#FUZZY PART------------------------------------------------------

probs = torch.softmax(logits, dim = 1)  # [batc_size, 7]
#CRIAR ALGO GENERICO QUE DÉ PARA TODAS AS EMOÇOEES SEM TER QUE ESCREVR UMA POR UMA
'Obs: mais avançado seria usar trapezoidal ou triangular membership functions, mas para o projeto de aula isto já é suficiente para interpretabilidade'

def fuzzy_happy(prob):
    if prob < 0.3:
        return "slightly happy"
    elif prob < 0.7:
        return "moderately happy"
    else:
        return "highly happy"

# probs → saída softmax da CNN
softmax_probs = torch.softmax(logits, dim=1).cpu().numpy()[0]  # exemplo de 1 imagem

labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
fuzzy_labels = []

for i, label in enumerate(labels):
    p = softmax_probs[i]
    if label == 'Happy':
        fuzzy_labels.append(fuzzy_happy(p))
    # repetir para outras emoções com suas funções fuzzy

print(fuzzy_labels)



: 